<a href="https://colab.research.google.com/github/simonsanvil/ECG-classification-MLH/blob/master/notebooks/simon_ENCASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTING REQUIRED LIBRARIES
from google.colab import drive
from collections import Counter
from shutil import copyfile
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import zipfile
import os, sys

%load_ext autoreload
%matplotlib inline

In [2]:
# DOWNLOAD AND LOAD PREPROCESSED TRAINING DATA

%cd /content

!gdown --id 1j7Yk0-W9uonQiyyG2B9ZAw5jjZA-UeMS

import pickle
data_dict = pickle.load(open("challenge2017.pkl","rb"))
data = data_dict["data"]
labels = data_dict['label']
df = pd.DataFrame(data,columns=["ts"])\
      .assign(label = pd.Series(labels))#.replace({"N":"Normal","A":"Atrial Fibrillation","O":"Other Rhythm","~":"Noise"}))
df

/content
Downloading...
From: https://drive.google.com/uc?id=1j7Yk0-W9uonQiyyG2B9ZAw5jjZA-UeMS
To: /content/challenge2017.pkl
100% 167M/167M [00:00<00:00, 180MB/s]


,ts,label
0,"[-127, -162, -197, -229, -245, -254, -261, -26...",N
1,"[128, 157, 189, 226, 250, 257, 262, 265, 268, ...",N
2,"[56, 73, 85, 93, 100, 107, 113, 117, 118, 117,...",N
3,"[519, 619, 723, 827, 914, 956, 955, 934, 920, ...",A
4,"[-188, -239, -274, -316, -356, -374, -380, -38...",A
...,...,...
8523,"[-104, -127, -144, -154, -160, -164, -168, -17...",N
8524,"[-121, -197, -288, -390, -496, -600, -693, -77...",O
8525,"[-202, -235, -272, -305, -325, -339, -344, -34...",N
8526,"[-227, -297, -374, -449, -505, -527, -539, -54...",N


In [3]:
df.label.value_counts()

N    5076
O    2415
A     758
~     279
Name: label, dtype: int64

In [4]:
# Download the repository of the model
%cd /content/
!git clone https://github.com/hsd1503/ENCASE

%cd /content/ENCASE/
!bash setup.sh

sys.path.append("/content/ENCASE/code")

# edit files that have dependency errors
%aimport -FeatureExtract -preprocess_data -dill
%autoreload 1 

with open("code/FeatureExtract.py","r+") as f:
  lines = f.readlines()
lines[22] = "#from BasicCLF import MyXGB\n"
lines[23] = "#from BasicCLF import MyLR\n"
lines[24] = "#from BasicCLF import MyRF\n"
with open("code/FeatureExtract.py","w+") as f:
  f.writelines(lines)

with open("code/preprocess_data.py","r+") as f:
  lines = f.readlines()
lines[11] = "#import hickle\n"
with open("code/preprocess_data.py","w+") as f:
  f.writelines(lines)

# make code/ a module
with open("code/__init__.py","w"):
  pass

%aimport code

/content
fatal: destination path 'ENCASE' already exists and is not an empty directory.
/content/ENCASE
Processing ./dill-0.2.6.zip
  Created wheel for dill: filename=dill-0.2.6-py3-none-any.whl size=53713 sha256=42e1c959d7aa16ec6c597740576824fd6e6865cc1baf77667a51043a10d67f55
  Stored in directory: /root/.cache/pip/wheels/9e/04/38/7888edb9723f1a1689c9b6b5e1741ce93a9c237b1116ba3944
Successfully built dill
  Attempting uninstall: dill
    Found existing installation: dill 0.2.6
    Uninstalling dill-0.2.6:
      Successfully uninstalled dill-0.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.2.6 which is incompatible.


In [5]:
with open('model/v2.5_xgb5_all_v2.pkl', 'rb') as f:
  model = pickle.load(f)
model

In [ ]:
# model.predict(df["ts"].to_numpy()[0])